In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from lp_relax.funcs.lp_relax import (
    _get_identification_kwargs,
    solve_lp_convex,
)

In [ ]:
beta_grid = np.sort(np.concatenate((np.zeros(1), np.linspace(-0.2, 0.2, 3))))

shape_constraints = {"shape_constraints": ("decreasing", "decreasing")}


s_to_plot = [1, 2, 4, 10, 100]

In [ ]:
def solve_model_for_grid_and_slope(s: float, beta_grid: np.ndarray) -> dict:
    res = {}

    for beta in beta_grid:
        identification_kwargs = _get_identification_kwargs(
            beta=beta,
            bfunc_type="constant",
            shape_constraints=shape_constraints,
            idestimands="idlate",
        )

        res[beta] = solve_lp_convex(
            beta=beta,
            algorithm="ipopt",
            constraint_type="poly",
            s=s,
            identification_kwargs=identification_kwargs,
        )

    return res

In [ ]:
sols_by_s = [solve_model_for_grid_and_slope(s, beta_grid) for s in s_to_plot]

In [ ]:
# Put into single df
data = pd.concat(
    [
        pd.DataFrame(
            {
                "s": s,
                "beta": list(res.keys()),
                "lp": [res[beta]["lp"] for beta in res],
                "convex": [res[beta]["convex"] for beta in res],
            }
        )
        for s, res in zip(s_to_plot, sols_by_s, strict=False)
    ]
)

data

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data[data["s"] == data["s"].unique()[0]]["beta"],
        y=data[data["s"] == data["s"].unique()[0]]["lp"],
        mode="lines",
        name="LP",
    )
)

for s in s_to_plot:
    data_select = data[data["s"] == s]
    fig.add_trace(
        go.Scatter(
            x=data_select["beta"],
            y=data_select["convex"],
            mode="lines",
            name=f"Convex, s={s}",
        )
    )
fig.show()